### Final Script

In [81]:
import os
import pandas as pd
import pymysql

symptomsDf = pd.read_csv('./symptom_product.csv')
diseasesDf = pd.read_csv('./disease_product.csv')

csvs = [symptomsDf, diseasesDf]

def get_total_dict(read_csv):
    category_product_dict = dict()
    for csv_file in read_csv:
        categories =list(filter(lambda s: not pd.isna(s), csv_file['분류']))
        product_id_semi = csv_file['상품 id'].tolist()
        get_category_product_dict(categories, product_id_semi, category_product_dict)
    return category_product_dict

def get_category_product_dict(categories, product_id_semi, category_product_dict):
    if len(product_id_semi) % 12 == 0:
        for product_id in range(0, len(product_id_semi), 12):
            category_product_dict[categories[product_id // 12]] = product_id_semi[product_id: product_id + 12]
        return category_product_dict

def get_total_result(result_dict):
    result = []
    i = 1
    for key, value in result_dict.items():
        for prod_id in value:
            result.append([i, key, prod_id])
            i += 1
    return result

total_result_dict = get_total_dict(csvs)
total_result_list = get_total_result(total_result_dict)
df_total_result = pd.DataFrame(total_result_list)
df_total_result.to_csv("product_category.csv", index=False)


### RDS Connection

In [52]:
# def get_database_connection():
#     return pymysql.connect(
#         user='was-petcare-write',
#         passwd='cq#U-4cpZCYFHpJS',
#         host='localhost',
#         db='fitpetdoc',
#         port=5306,
#         charset='utf8mb4'
#     )
def fetch_result():
    # global cursor, sql

    def get_database_connection():
        return pymysql.connect(
            user='was-petcare-write',
            passwd='cq#U-4cpZCYFHpJS',
            host='localhost',
            db='fitpetdoc',
            port=5306,
            charset='utf8mb4'
        )

    cursor = get_database_connection().cursor()
    sql = "select csc.name, csc.id, cs.name, cs.id from core_symptom cs inner join core_symptomcategory csc on csc.id = cs.category_id"
    cursor.execute(sql)
    return cursor.fetchall()

# print('카테고리', '카테고리_아이디', '증상/질병', '증상/질병_아이디', '상품_아이디', 'tag')
# id = 1
# result_list = []
# fetchall = fetch_result()
# category_product_id_dict = get_category_product_id()
# for i in fetchall:
#     try:
#         category = i[0]
#         for product_id in category_product_id_dict[category]:
#             result_list.append([id, i[0], i[1], i[2], i[3], product_id, 'symptom'])
#             id += 1
#     except KeyError:
#         pass
#
# index_df = [i for i in range(1, len(result_list) + 1)]
# df = DataFrame(result_list,
#                columns=['id', 'category', 'category_id', 'symptoms_disease', 'symptom_disease_id', 'product_id', 'tag'],
#                index=index_df)
# df

In [58]:
# # 여기서 aurora db 커넥션 맺을 수 있나?
# """
# 증상 카테고리 별 상품 아이디 12개를 할당
# review_product 테이블
# - category 로 조회
# -> category 에 속하는 symptom/disease 반환
# -> 해당 symptom/disease 마다 열두개의 12할당.(여기가 어렵다. 어떻게 하나 짜리 row 를 12개로 복제하지)
# """
#
# # What to do
#
#
# # 터널링으로 접근하기 때문에 호스트는 localhost 지정한다.
# rds_connector = pymysql.connect(
#     user='was-petcare-write',
#     passwd='cq#U-4cpZCYFHpJS',
#     host='localhost',
#     db='fitpetdoc',
#     port=5306,
#     charset='utf8mb4'
# )
#
# curs = rds_connector.cursor()
# sql = "select * from review_product where tag = 'symptom' order by category, symptom_disease limit 5"
#
# curs.execute(sql)
# review_product_all = curs.fetchall()
#
# result = pd.DataFrame(review_product_all)
# result

#id, category_id, category, symptom_disease_id, symptom_disease, product_id, tag

In [59]:
# cursor = rds_connector.cursor()
# sql = "select csc.name, csc.id, cs.name, cs.id from core_symptom cs inner join core_symptomcategory csc on csc.id = cs.category_id"
# cursor.execute(sql)
# fetchall = cursor.fetchall()
# # print('카테고리', '카테고리_아이디', '증상/질병', '증상/질병_아이디', '상품_아이디', 'tag')
# id = 1
# result_list = []
# for i in fetchall:
#     try:
#         for product_id in symptom_product_id_dict[i[0]]:
#             result_list.append([id, i[0], i[1], i[2], i[3], product_id, 'symptom'])
#             id += 1
#     except KeyError:
#         pass
#
# index_df = [i for i in range(1, len(result_list) + 1)]
# df = DataFrame(result_list,
#                columns=['id', 'category', 'category_id', 'symptoms_disease', 'symptom_disease_id', 'product_id', 'tag'],
#                index=index_df)
# df

### Markdowntownbaby


1. 지금은 이미 증상/질환이 12개씩 생성된 상태
2. 이걸 다 날리고 새로 만드는게 나을까, 아니면 기존것에서 업데이트 치는게 낫겠니.

3. 일단 기존 테이블은 그대로 두고, 별도의 테이블 생성해서 테스트하자.
4.




In [11]:
result_ = ['눈',
           '코/호흡기',
           '치아구강',
           '다리',
           '피부',
           '소화기배변',
           '생식기비뇨',
           '행동변화',
           '신체변화',
           '기타',
           '증상없음',
           '코',
           '호흡기',
           '귀',
           '눈',
           '호흡기',
           '치아구강',
           '뼈관절',
           '피부',
           '비뇨기',
           '소화기',
           '생식기',
           '심장혈관',
           '내분비계',
           '뇌신경',
           '기타',
           '귀',
           '검진접종',
           '질환없음']

for r in set(result_):
    print(r)

질환없음
내분비계
코/호흡기
기타
소화기배변
호흡기
뇌신경
생식기비뇨
피부
비뇨기
신체변화
행동변화
다리
생식기
검진접종
치아구강
코
소화기
눈
귀
증상없음
뼈관절
심장혈관
